# Example of validating the mapping file (provenance) using schema

In [1]:
## CX: allows multiple lines of code to print from one code block
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
import jsonschema
import pathlib
import jsonref
import yaml

In [2]:
pathlib.Path.cwd()

PosixPath('/Users/jay/Desktop/biothings_explorer/jupyter notebooks/CX_WIPs/Draft_Metadata')

## loading schema

In [3]:
yaml_schema = pathlib.Path.cwd().joinpath("draft6_schema_mappingFile.yaml")
with open(yaml_schema) as file:
    schema_from_yaml = yaml.load(file, Loader=yaml.SafeLoader)
    schema_from_yaml = json.dumps(schema_from_yaml, indent=2)
    schema_from_yaml = jsonref.loads(schema_from_yaml)

In [4]:
schema_from_yaml.keys()
schema_from_yaml['description']

dict_keys(['title', '$schema', 'type', 'description', 'properties', 'definitions'])

'Contains the mapping between possible values in the response fields (as string keys) to info in the desired format. Will be used often for provenance'

In [5]:
schema_from_yaml['properties'].keys()

dict_keys(['provenanceMapping'])

In [6]:
schema_from_yaml['properties']['provenanceMapping']['patternProperties']\
['.']['items']['properties'].keys()

dict_keys(['name', 'sourceType', 'version', 'versionType', 'method', 'sourceReferences', 'descriptiveInfo', 'sourceContext'])

In [7]:
schema_from_yaml['properties']['provenanceMapping']['patternProperties']\
['.']['items']['properties']['sourceReferences']['properties']['websites']\
['properties']['value']['oneOf']

[{'type': 'array', 'minItems': 1, 'items': {'type': 'string'}},
 {'type': 'string'}]

## checking an example

In [8]:
yaml_example = pathlib.Path.cwd().joinpath("draft6_example_disgenetDGGD_provenance.yaml")
with open(yaml_example) as file:
    example_from_yaml = yaml.load(file, Loader=yaml.SafeLoader)
    example_from_yaml = json.dumps(example_from_yaml, indent=2)
    example_from_yaml = jsonref.loads(example_from_yaml)

In [9]:
example_from_yaml['provenanceMapping'].keys()

dict_keys(['LHGDN', 'BEFREE', 'HPO', 'UNIPROT', 'CLINVAR', 'GWASCAT', 'GWASDB', 'MGD', 'RGD', 'CTD_mouse', 'CTD_rat', 'CGI', 'CLINGEN', 'GENOMICS_ENGLAND', 'CTD_human', 'PSYGENET', 'ORPHANET'])

In [10]:
example_from_yaml['provenanceMapping']['CTD_human'][0]

{'name': 'MyDisease.info API',
 'sourceType': 'service',
 'version': '2020-10-26',
 'versionType': 'date_last_updated',
 'method': 'ingest',
 'descriptiveInfo': 'Downloaded the DisGeNET ALL gene-disease-pmid associations file. Merged gene-disease associations that differed only in the pmid value (so each association now had a list of ids in the pmid column).'}

In [11]:
example_from_yaml['provenanceMapping']['CTD_human'][2]

{'name': 'CTD',
 'sourceType': 'knowledgebase',
 'method': 'manual_curation',
 'sourceContext': {'taxonSpecific': {'value': 'NCBITaxon:9606'}},
 'sourceReferences': {'websites': {'value': 'http://ctdbase.org/about/'}},
 'descriptiveInfo': 'The curated gene-disease associations from CTD are categorized as marker/mechanism and/or therapeutic (when a person looks up the genes associated with a specific disease in CTD, this info on what associations are manually curated is given).'}

## validate example against schema

In [12]:
jsonschema.validate(instance=example_from_yaml, schema=schema_from_yaml)
## YAYAYAYAYAY this means it validated!!!!!!!

### error: if a required field is missing

In [13]:
## but what if it's a fluke?? 
## remove a required key from one association 
removed1 = example_from_yaml['provenanceMapping']['CTD_human'][2].pop('name')

## then try to validate, this 
jsonschema.validate(instance=example_from_yaml, schema=schema_from_yaml)
## so it works in catching the error yayyyyyyyy. too bad it doesn't tell you which entry had the error in it...

ValidationError: 'name' is a required property

Failed validating 'required' in schema['properties']['provenanceMapping']['patternProperties']['.']['items']:
    {'additionalProperties': False,
     'properties': {'descriptiveInfo': {'description': 'Long-text '
                                                       'description of '
                                                       'what the source '
                                                       'did to create '
                                                       'associations. Try '
                                                       'to include enough '
                                                       'detail that a '
                                                       'reader would '
                                                       'understand what '
                                                       'happened',
                                        'type': 'string'},
                    'met

In [14]:
example_from_yaml['provenanceMapping']['CTD_human'][2]['name'] = removed1

In [15]:
jsonschema.validate(instance=example_from_yaml, schema=schema_from_yaml)
## and it's back and accepted again 

## Export JSON files for the yamls

In [16]:
json_schema_path = pathlib.Path.cwd().joinpath("draft6_schema_mappingFile.json")
with open(json_schema_path, "w") as file:
    json.dump(schema_from_yaml, file, indent=2)

In [17]:
json_example_path = pathlib.Path.cwd().joinpath("draft6_example_disgenetDGGD_provenance.json")
with open(json_example_path, "w") as file:
    json.dump(example_from_yaml, file, indent=2)